In [2]:
import json
import os
import pandas as pd
import glob
import pandasgui as pdg

print(pdg.__version__)

0.2.13


In [13]:
dataset = ["OpenAPS", "OPENonOH"]
outdir = "/home/reinhold/Daten/OPEN/"
outfilename = f"duplicates_{dataset[0]}_{dataset[1]}.csv"

indir = [f"{outdir}/{d}_Data/csv_per_day/" for d in dataset]
infilename = [f"entries_{d}.csv" for d in dataset]

df = [pd.read_csv(os.path.join(indir[i], infilename[i]), header=0, parse_dates=[1], index_col=0) for i in range(2)]
df[0]["date"] = pd.to_datetime(df[0]["date"],format="%Y-%m-%d")
out = df[0].merge(df[1], left_on="date", right_on="date", how="inner", suffixes=("_OpenAPS", "_OPENonOH"))

print(df[0].columns)
for col in df[0].columns:
    if not "sgv" in col: continue
    out["diff_" + col] = 2*(out[col + "_OpenAPS"] - out[col + "_OPENonOH"])/(out[col + "_OpenAPS"] + out[col + "_OPENonOH"])

df2 = out[(out.diff_sgv_mean*out.diff_sgv_mean< 1e-4) & (out.diff_sgv_std*out.diff_sgv_std< 1e-4) & (out.diff_sgv_min*out.diff_sgv_min< 1e-4) & (out.diff_sgv_max*out.diff_sgv_max< 1e-4) & (out.diff_sgv_count*out.diff_sgv_count< 1e-4)]
df3 = df2[["user_id_OpenAPS", "user_id_OPENonOH", "date", "diff_sgv_mean", "diff_sgv_std", "diff_sgv_min", "diff_sgv_max", "diff_sgv_count", "second_id_OPENonOH", "filename_OPENonOH", "filename_OpenAPS"]].sort_values(by=["user_id_OpenAPS", "user_id_OPENonOH", "date"])
#df3 = df3[[]]

# group by statement has to return exactly one line, otherwise the selection criteria on diff_sgv_*-variables need to be tightened.

gui = pdg.show(df3)

df3.to_csv(os.path.join(outdir, outfilename))
print(os.path.join(outdir, outfilename) + " created")

df4 = df3.groupby(['user_id_OpenAPS', 'user_id_OPENonOH']).agg("count")
columns = []
print(df4.columns)
for col in df4.columns:
    columns.append(f"{col[0]}_{col[1]}")
df4.columns = columns
df4.fillna(value=0, inplace=True)  #
df4.reset_index(inplace=True)
df4.info()
print(df4[['user_id_OpenAPS', 'user_id_OPENonOH','date_count']])


PandasGUI INFO — pandasgui.gui — Opening PandasGUI


Index(['date', 'sgv_mean', 'sgv_std', 'sgv_min', 'sgv_max', 'sgv_count',
       'filename', 'user_id', 'second_id'],
      dtype='object')
/home/reinhold/Daten/OPEN/duplicates_OpenAPS_OPENonOH.csv created
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   user_id_OpenAPS   7 non-null      int64
 1   user_id_OPENonOH  7 non-null      int64
 2   d_a               7 non-null      int64
 3   d_i               7 non-null      int64
 4   d_i               7 non-null      int64
 5   d_i               7 non-null      int64
 6   d_i               7 non-null      int64
 7   d_i               7 non-null      int64
 8   s_e               7 non-null      int64
 9   f_i               7 non-null      int64
 10  f_i               7 non-null      int64
dtypes: int64(11)
memory usage: 744.0 bytes


KeyError: "['date_count'] not in index"